In [2]:
model = "llama3.2:1B"

#### Task 1: Simple Chain with Retrieval

**Objective:**

Implement a simple RAG chain with ChatOllama, HuggingFaceEmbeddings and Chroma.

Process:

1. Retrieve documents from chroma db based on query
2. Invoke chain with retrieved documents as input

**Task Description:**

- load llm model via ollama
- load embedding model via ollama with `ollama pull pull bge-m3` (if not yet done)
- create chroma db client
- create prompt template for summarization
- create simple chain with following steps: retrieved documents, prompt, model, output parser
- create query and perform similarity search with a query
- invoke chain and pass retrieved documents to the chain

**Useful links:**

- [RAG with Ollama](https://python.langchain.com/v0.2/docs/tutorials/local_rag/)


In [3]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=model)

In [4]:
from langchain_ollama import OllamaEmbeddings

embedding_model = OllamaEmbeddings(
    model="bge-m3",
)

In [20]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

collection = client.get_or_create_collection("ai_model_book")

vector_db_from_client = Chroma(
    client=client,
    collection_name="ai_model_book",
    embedding_function=embedding_model,
)

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

In [22]:
search_query = "Types of Machine Learning Systems"

docs = vector_db_from_client.similarity_search(search_query)

print(docs)

[Document(metadata={'page': 33, 'source': './AI_Book.pdf'}, page_content='Types of Machine Learning Systems\nThere are so many different types of Machine Learning systems that it is useful to\nclassify them in broad categories based on:\n•Whether or not they are trained with human supervision (supervised, unsuper‐\nvised, semisupervised, and Reinforcement Learning)\n•Whether or not they can learn incrementally on the fly (online versus batch\nlearning)\n•Whether they work by simply comparing new data points to known data points,\nor instead detect patterns in the training data and build a predictive model, much\nlike scientists do (instance-based versus model-based learning)\nThese criteria are not exclusive; you can combine them in any way you like. For\nexample, a state-of-the-art spam filter may learn on the fly using a deep neural net‐\nwork model trained using examples of spam and ham; this makes it an online, model-\nbased, supervised learning system.\nLet’s look at each of these

In [23]:
chain.invoke(docs)

'Here are the main themes from the retrieved documents:\n\n1. **Classification of Machine Learning Systems**: The document discusses the different types of machine learning systems based on four criteria:\n\t* Supervised/unsupervised learning (supervised, unsupervised, semisupervised, and Reinforcement Learning)\n\t* Incremental online vs. batch learning\n\t* Instance-based vs. model-based learning\n2. **Supervised Learning**: The document explains the concept of supervised learning, where the training data includes desired solutions or labels, as seen in Figure 1-5.\n3. **Batch and Online Learning**: The document highlights the difference between batch learning (training using all available data offline) and online learning (learning incrementally from a stream of incoming data).\n4. **Model-based vs. Instance-based Learning**: The document distinguishes between model-based learning (building a predictive model based on examples) and instance-based learning (making predictions based o

#### Task 2: Q&A with RAG

**Objective:**

Implement a Q/A retrieval chain with ChatOllama, HuggingFaceEmbeddings and Chroma

**Task Description:**

- create RAG-Q/A prompt template
- create retriever from vector db client (instead of manually passing in docs, we automatically retrieve them from our vector store based on the user question)
- create simple chain with following steps: retriever, formatting retrieved docs, user question, prompt, model, output parser
- create question for Q/A retrieval chain
- invoke chain and with question

**Useful links:**

- [RAG with Ollama](https://python.langchain.com/v0.2/docs/tutorials/local_rag/)


In [24]:
from langchain_core.runnables import RunnablePassthrough

prompt_template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(prompt_template)

retriever = vector_db_from_client.as_retriever()

qa_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [25]:
qa_rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0xfffef4480910>)
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\n<context>\n{context}\n</context>\n\nAnswer the following question:\n\n{question}"))])
| ChatOllama(model='llama3.2', _client=<ollama._client.Client object at 0xfffef66a7450>, _async_client=<ollama._client.AsyncClient object at 0xfffef796e010>)
| StrOutputParser()

In [26]:
question = "What is supervised learning?"

qa_rag_chain.invoke(question)

'Supervised learning is a type of machine learning where the training data includes the desired solutions, called labels. In this approach, the algorithm learns from labeled examples to make predictions on new, unseen data. The goal is to map inputs to outputs based on the patterns learned from the labeled training set.'